<a href="https://colab.research.google.com/github/ssp22/485Design-Project/blob/master/Python%20%2B%20Derivations/optimization_final_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [38]:
# Define parameters
  dP = 40 # bar
  dPi = 5.7 # bar
  R = 0.1 # at 40 bar, 5C
  k_w = 1.25 * 10 ** (-5) # g / (cm^2 s bar)
  diff_P = dP - dPi
  X = 0.05 # split ratio
  C_Et0 = 0.0395 # weight percent
  M_f = 1000 # g / s # flow out of tank
  t = np.linspace(0,10000,10000)
  Product = 6 * 10**4
  Max = Product
  Limit = 6.1 * 10**4
  A = 953000 # cm^2 # membrane area
  Parameters = [1,2,3,4,5]

for i in range(650000,840000,5000):
    A = i
    for j in range(1600000,1900000,10000):
        m_0 = j # g # initial batch feed
        for k in range(500,2000,100):
          M_f = k
          # solve ODE
          C = odeint(dCdt2,C_Et0,t)
          count = 0
          val= 10
          if (np.amin(C)<=0.00395):
            while (val >= 0.00395):
              val = C[count]
              count += 1
            # break
          if (val <= 0.00395):
            Total = 365*20*3600/t[count]*m_0/ (29.8 * 3968)
            if (Total > Max and Total <= Limit):
              Max = Total
              Parameters[0] = i
              Parameters[1] = j
              Parameters[2] = k
              Parameters[3] = t[count]
              Parameters[4] = val
 
print('membrane area = ', Parameters[0],'cm2')
print('batch feed = ',Parameters[1],' g')
print('flow out of tank = ',Parameters[2],'g/s')
print('time for one batch',Parameters[3] ,'s')
print('final conc = ', Parameters[4])
print('total product= ', Max,' BPY')

membrane area =  795000 cm2
batch feed =  1800000  g
flow out of tank =  1500 g/s
time for one batch 6564.656465646564 s
final conc =  [0.00394992]
total product=  60939.36639676434  BPY


In [0]:
def dCdt2(C_Et,t):
    # Define flow rates
    M_p1 = A * k_w * diff_P * (1 + C_Et * (1-R)) 
    M_r1 = M_f - M_p1
    if (M_r1 < 0):
      return 0
    C_r1 = (M_f * C_Et - M_p1 * (1-R) * C_Et) / M_r1
    M_p2 = A * k_w * diff_P * (1 + C_r1 * (1-R))
    M_r2 = (1-X) * M_r1 - M_p2
    if (M_r2 < 0):
      return 0
    C_r2 = ((1-X)*M_r1*C_r1 - M_p2 * (1-R) * C_r1) / M_r2
    dCdt = (M_r1 * X * C_r1 + M_r2 * C_r2 - M_f * C_Et) / m_0
    M_water = M_p1 + M_p2
    return dCdt